# Подход к оценке влияния социальных денежных трансфертов на устойчивый рост доходов и снижение бедности

### Какие денежные выплаты предназначены для поддержки семей с детьми?

1. Выплаты по беременности и рождению ребенка:
    - Единовременное пособие женщинам, вставшим на учет в медицинских учреждениях в ранние сроки беременности; 
    - Декретные выплаты по беременности и родам (БиР);
    - Единовременное пособие при рождении (усыновлении) ребенка;
    - Ежемесячное пособие по уходу за ребенком до 1,5 лет;
2. Ежемесячная выплата малоимущим семьям (менее 2 ПМ на 1 члена) в связи с рождением (усыновлением) первого, второго ребенка до 3-х лет;
3. Ежемесячные выплаты на третьего и последующих детей;
4. Ежемесячные выплаты малоимущим семьям (менее 2 ПМ на 1 члена) на детей в возрасте от 3 до 7 лет;

В данном исследовании мы опустим оценку влияния выплат по беременности и рождению ребенка:
Очевидно, что единовременные пособия не окажут значимого влияния на устойчивый рост доходов населения, в том числе за счёт небольшого размера выплат (675 рублей за постановку на учет и 18 тыс. при рождении). Декретные выплаты по беременности и родам, а также ежемесячное пособие по уходу за ребенком выплачиваются ФСС и напрямую зависят от размера заработной платы родителей, следовательно, её сложно будет оценить. 

In [2]:
#%% Load environment

import pandas as pd
import numpy as np

### Merging individuals to households and poverty lines 

In [3]:
#columns to pick

clms_ind = ['H00_02', #Код субъекта РФ
            'H00_06', #Код домохозяйства
            'H01_00', #Код индивидуума
            'H01_02'] #Число исполнившихся лет

clms_hHolds = ['H00_02', #Код субъекта РФ
               'H00_06', #Код домохозяйства
               'R_H_DOX_SOVK_BEZ', #Совокупный доход домохозяйства
               'KVZV']   #Вес домохозяйства в общей численности населения

In [4]:
#Reading individual`s data

individuals = pd.read_csv('/Users/anastasiia/Downloads/СП/БАЗЫ/ВНДН/ДОХОДЫ2019_IND_за 2018.csv', 
                          sep = ';', 
                          usecols = clms_ind, 
                          decimal = ',')
individuals.columns = ['IND_ID', 'AGE', 'REGION_ID', 'HHOLD_ID']
individuals.head()

,IND_ID,AGE,REGION_ID,HHOLD_ID
0,2,36,45,10714
1,1,27,45,2376
2,1,59,82,537
3,1,36,86,231
4,2,35,65,736


In [5]:
#Check children under 3 years old 
individuals['UNDER_3'] = 0
individuals.loc[individuals.AGE < 3, 'UNDER_3'] = 1

#Check children from 3 to 7 years old
individuals['FROM_3_TO_7'] = 0
individuals.loc[(individuals.AGE >= 3) & (individuals.AGE < 7), 'FROM_3_TO_7'] = 1

#Check children under 18
individuals['UNDER_18'] = 0
individuals.loc[individuals.AGE < 18, 'UNDER_18'] = 1
        
individuals.head()

,IND_ID,AGE,REGION_ID,HHOLD_ID,UNDER_3,FROM_3_TO_7,UNDER_18
0,2,36,45,10714,0,0,0
1,1,27,45,2376,0,0,0
2,1,59,82,537,0,0,0
3,1,36,86,231,0,0,0
4,2,35,65,736,0,0,0


In [6]:
#Group by region_id and hhold_id

indGrouped = (individuals.groupby(['REGION_ID', 'HHOLD_ID'])
     .agg({'IND_ID':'count', 'UNDER_3': 'sum', 'FROM_3_TO_7': 'sum','UNDER_18': 'sum'})
     .reset_index()
     .rename(columns={'IND_ID':'HHOLD_SIZE'}))

indGrouped.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18
0,1,2,2,0,0,0
1,1,3,3,0,0,1
2,1,4,2,0,0,0
3,1,5,2,0,0,0
4,1,6,2,0,0,0


In [7]:
#Reading households data

hHolds = pd.read_csv('/Users/anastasiia/Downloads/СП/БАЗЫ/ВНДН/ДОХОДЫ2019_HHOLD_за 2018.csv', 
                        sep = ';', 
                        usecols = clms_hHolds,
                        decimal = ',')
hHolds.columns = ['REGION_ID', 'HHOLD_ID', 'INCOME', 'HHOLD_WEIGHTS']

hHolds.head()

,REGION_ID,HHOLD_ID,INCOME,HHOLD_WEIGHTS
0,3,201,3.200000e+06,47339.387260
1,4,18,3.234385e+06,1403.170152
2,8,499,3.777361e+06,630.227333
3,10,490,7.546790e+06,1852.445984
4,26,193,9.520779e+05,234.971610


In [8]:
#Reading data for the poverty lines grouped by category 

povLine = pd.read_excel('/Users/anastasiia/Downloads/ПМ.xlsx')

povLine = povLine[povLine.PERIOD == '2Q'] #referring to the 2nd quartal of the year that preceed the year of application

povLine.head()

,REGION_ID,REGION,YEAR,PERIOD,CHILDREN_PL,PENSIONERS_PL,WORKING_AGE_PL
85,31,Белгородская область,2016,2Q,8150,6822,8826
86,32,Брянская область,2016,2Q,9068,7659,9889
87,33,Владимирская область,2016,2Q,9495,7856,10183
88,36,Воронежская область,2016,2Q,8277,6938,8990
89,37,Ивановская область,2016,2Q,9697,8026,10429


In [9]:
#Join 3 tables in 1 and name it vndn

vndn = pd.merge(pd.merge(indGrouped, hHolds, how = 'left', on=['REGION_ID', 'HHOLD_ID']), 
                povLine[['REGION_ID', 'CHILDREN_PL', 'WORKING_AGE_PL']], how = 'inner', on = ['REGION_ID'])

vndn.tail()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL
52291,92,11140,4,0,1,2,456107.260000,3855.251629,10375,10503
52292,92,11142,1,0,0,0,231710.280000,477.901485,10375,10503
52293,92,11146,4,2,0,2,660247.900689,1741.416084,10375,10503
52294,92,11148,1,0,0,0,142792.000000,365.767322,10375,10503
52295,92,11149,1,0,0,0,190538.280000,477.901485,10375,10503


In [10]:
#Calculate average per capita monthly household income

vndn['PER_CAPITA_HHOLD_MONTHLY_INCOME'] = vndn.INCOME/(vndn.HHOLD_SIZE*12)

vndn.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000


In [11]:
#Create a new column that highlights low-income households

vndn['UNDER_2PL'] = 0
vndn.loc[vndn.PER_CAPITA_HHOLD_MONTHLY_INCOME < 2*vndn.WORKING_AGE_PL, 'UNDER_2PL'] = 1

vndn.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0


In [12]:
#Create a new column that highlights poor households

vndn['UNDER_1PL'] = 0
vndn.loc[vndn.PER_CAPITA_HHOLD_MONTHLY_INCOME < vndn.WORKING_AGE_PL, 'UNDER_1PL'] = 1

vndn.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0


In [13]:
#For low-income households calculate deficit of income

vndn['MONTHLY_DEFICIT'] = 0
vndn.loc[vndn.UNDER_1PL == 1, 'MONTHLY_DEFICIT'] = abs(vndn.INCOME/12 - vndn.HHOLD_SIZE*vndn.WORKING_AGE_PL)

vndn.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0


In [14]:
#Weight monthly deficit by coefficients that extrapolate sample results to the population

vndn['MONTHLY_DEFICIT_WEIGHTED'] = 0
vndn.loc[vndn.UNDER_1PL == 1, 'MONTHLY_DEFICIT_WEIGHTED'] = vndn.MONTHLY_DEFICIT*vndn.HHOLD_WEIGHTS

vndn.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0


In [15]:
#Monthly household deficit for the sample 

print("Ежемесячный дефицит доходов населения равен", 
    round(sum(vndn.MONTHLY_DEFICIT_WEIGHTED)/10**6),
      "млн рублей. Всего к малоимущим (доход ниже ПМ) относятся",
    round(sum(vndn.UNDER_1PL*vndn.HHOLD_WEIGHTS)),
      "домохозяйств, в которых проживает (",
    round(sum(vndn.UNDER_1PL*vndn.HHOLD_WEIGHTS*vndn.HHOLD_SIZE)),
      ") человек.",
    round(sum(vndn.UNDER_1PL*vndn.HHOLD_WEIGHTS)/sum(vndn.HHOLD_WEIGHTS)*100, 2),
      "% от всех домохозяйств")

Ежемесячный дефицит доходов населения равен 47950 млн рублей. Всего к малоимущим (доход ниже ПМ) относятся 4045970 домохозяйств, в которых проживает ( 14622144 ) человек. 7.73 % от всех домохозяйств


### Ежемесячная выплата малообеспеченным семьям (менее 2 ПМ на 1 члена) в связи с рождением (усыновлением) первого, второго ребенка до 3-х лет 

In [16]:
#Define the target audience

underThree = vndn[(vndn.UNDER_2PL == 1) & (vndn.UNDER_18 < 3) & (vndn.UNDER_3 != 0)]

underThree.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED
127,1,175,4,1,0,1,319082.363792,932.562071,8866,9302,6647.549246,1,1,10617.803017,9.901760e+06
174,1,233,3,1,0,1,178647.520689,1995.127399,8866,9302,4962.431130,1,1,13018.706609,2.597398e+07
270,1,379,4,1,0,2,130435.340000,925.854678,8866,9302,2717.402917,1,1,26338.388333,2.438552e+07
288,1,403,4,1,0,2,441636.754482,1712.882502,8866,9302,9200.765718,1,1,404.937127,6.936097e+05
319,1,445,4,1,0,1,348462.881724,792.798385,8866,9302,7259.643369,1,1,8169.426523,6.476708e+06


In [17]:
#Calculate transfer

underThree['MONTHLY_TRANSFER'] = underThree.UNDER_3 * underThree.CHILDREN_PL

underThree.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER
127,1,175,4,1,0,1,319082.363792,932.562071,8866,9302,6647.549246,1,1,10617.803017,9.901760e+06,8866
174,1,233,3,1,0,1,178647.520689,1995.127399,8866,9302,4962.431130,1,1,13018.706609,2.597398e+07,8866
270,1,379,4,1,0,2,130435.340000,925.854678,8866,9302,2717.402917,1,1,26338.388333,2.438552e+07,8866
288,1,403,4,1,0,2,441636.754482,1712.882502,8866,9302,9200.765718,1,1,404.937127,6.936097e+05,8866
319,1,445,4,1,0,1,348462.881724,792.798385,8866,9302,7259.643369,1,1,8169.426523,6.476708e+06,8866


In [18]:
#Calculate weighted transfer

underThree['MONTHLY_TRANSFER_WEIGHTED'] = underThree.MONTHLY_TRANSFER * underThree.HHOLD_WEIGHTS

underThree.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED
127,1,175,4,1,0,1,319082.363792,932.562071,8866,9302,6647.549246,1,1,10617.803017,9.901760e+06,8866,8.268095e+06
174,1,233,3,1,0,1,178647.520689,1995.127399,8866,9302,4962.431130,1,1,13018.706609,2.597398e+07,8866,1.768880e+07
270,1,379,4,1,0,2,130435.340000,925.854678,8866,9302,2717.402917,1,1,26338.388333,2.438552e+07,8866,8.208628e+06
288,1,403,4,1,0,2,441636.754482,1712.882502,8866,9302,9200.765718,1,1,404.937127,6.936097e+05,8866,1.518642e+07
319,1,445,4,1,0,1,348462.881724,792.798385,8866,9302,7259.643369,1,1,8169.426523,6.476708e+06,8866,7.028950e+06


In [19]:
#Calculate, what amount of deficit is left uncovered by social transfer

underThree['UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = 0
underThree.loc[underThree.MONTHLY_DEFICIT_WEIGHTED > underThree.MONTHLY_TRANSFER_WEIGHTED, 
               'UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = (underThree.MONTHLY_DEFICIT_WEIGHTED - 
                                                        underThree.MONTHLY_TRANSFER_WEIGHTED)

underThree.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED,UNCOVERED_MONTHLY_DEFICIT_WEIGHTED
127,1,175,4,1,0,1,319082.363792,932.562071,8866,9302,6647.549246,1,1,10617.803017,9.901760e+06,8866,8.268095e+06,1.633665e+06
174,1,233,3,1,0,1,178647.520689,1995.127399,8866,9302,4962.431130,1,1,13018.706609,2.597398e+07,8866,1.768880e+07,8.285179e+06
270,1,379,4,1,0,2,130435.340000,925.854678,8866,9302,2717.402917,1,1,26338.388333,2.438552e+07,8866,8.208628e+06,1.617689e+07
288,1,403,4,1,0,2,441636.754482,1712.882502,8866,9302,9200.765718,1,1,404.937127,6.936097e+05,8866,1.518642e+07,0.000000e+00
319,1,445,4,1,0,1,348462.881724,792.798385,8866,9302,7259.643369,1,1,8169.426523,6.476708e+06,8866,7.028950e+06,0.000000e+00


In [22]:
print ("Ежемесячный дефицит доходов семей с первыми и вторыми детьми до 3-лет, доход которых ниже 2 региональных ПМ, составляет", 
    round(sum(underThree.MONTHLY_DEFICIT_WEIGHTED)/10**6), 
       "млн рублей. Общий объём трансфертов равен",
    round(sum(underThree.MONTHLY_TRANSFER_WEIGHTED)/10**6), 
       "млн рублей. Пособие на первых и вторых детей до трёх лет снижает месячный дефицит доходов на", 
    round((sum(underThree.MONTHLY_DEFICIT_WEIGHTED) - sum(underThree.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/10**6), 
       "млн рублей, что составляет ", 
    round(100 - sum(underThree.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED)/sum(underThree.MONTHLY_DEFICIT_WEIGHTED)*100),
       "% от дефицита доходов целевой аудитории (семей с первыми и вторыми детьми до 3-х лет) и", 
    round((sum(underThree.MONTHLY_DEFICIT_WEIGHTED) - 
           sum(underThree.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/sum(vndn.MONTHLY_DEFICIT_WEIGHTED)*100, 2), 
       "% от дефицита доходов населения. Всего в России", 
    round(sum(underThree.UNDER_2PL*underThree.HHOLD_WEIGHTS)),
       "домохозяйств, которые могут получить этот вид пособия, в которых проживают",
    round(sum(underThree.UNDER_2PL*underThree.HHOLD_SIZE*underThree.HHOLD_WEIGHTS)), 
       "человек. Доходы",
    round(sum(underThree.UNDER_1PL*underThree.HHOLD_WEIGHTS)),
       "домохозяйств (", 
    round(sum(underThree.UNDER_1PL*underThree.HHOLD_SIZE*underThree.HHOLD_WEIGHTS)), " человек) ниже уровня ПМ.")

Ежемесячный дефицит доходов семей с первыми и вторыми детьми до 3-лет, доход которых ниже 2 региональных ПМ, составляет 7374 млн рублей. Общий объём трансфертов равен 18432 млн рублей. Пособие на первых и вторых детей до трёх лет снижает месячный дефицит доходов на 4906 млн рублей, что составляет  67 % от дефицита доходов целевой аудитории (семей с первыми и вторыми детьми до 3-х лет) и 10.23 % от дефицита доходов населения. Всего в России 1621223 домохозяйств, которые могут получить этот вид пособия, в которых проживают 6577428 человек. Доходы 547797 домохозяйств ( 2203876  человек) ниже уровня ПМ.


### Ежемесячная выплата на третьих и последующих детей до 3-х лет

In [23]:
#Define the target audience

manyChildren = vndn[(vndn.UNDER_18 > 2) & (vndn.UNDER_3 != 0)]

manyChildren.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00
164,1,223,5,1,0,3,306265.282758,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07
170,1,229,6,1,1,3,564214.583103,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06
348,1,493,6,1,2,5,242276.000000,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07
486,1,715,6,1,1,4,641663.724137,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07


In [24]:
#Define how many children will receive the tranfer

manyChildren['ALIGIBILITY'] = np.where((manyChildren.UNDER_18 - manyChildren.UNDER_3) >= 2, manyChildren.UNDER_3, 
         (np.where((manyChildren.UNDER_18 - manyChildren.UNDER_3) == 1, (manyChildren.UNDER_3 - 1), 
                   (manyChildren.UNDER_3 - 2))))

manyChildren.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,ALIGIBILITY
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,1
164,1,223,5,1,0,3,306265.282758,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07,1
170,1,229,6,1,1,3,564214.583103,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06,1
348,1,493,6,1,2,5,242276.000000,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07,1
486,1,715,6,1,1,4,641663.724137,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07,1


In [25]:
#Calculate transfer

manyChildren['MONTHLY_TRANSFER'] = manyChildren.ALIGIBILITY * manyChildren.CHILDREN_PL

manyChildren.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,ALIGIBILITY,MONTHLY_TRANSFER
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,1,8866
164,1,223,5,1,0,3,306265.282758,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07,1,8866
170,1,229,6,1,1,3,564214.583103,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06,1,8866
348,1,493,6,1,2,5,242276.000000,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07,1,8866
486,1,715,6,1,1,4,641663.724137,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07,1,8866


In [26]:
#Calculate transfer weighted

manyChildren['MONTHLY_TRANSFER_WEIGHTED'] = manyChildren.MONTHLY_TRANSFER * manyChildren.HHOLD_WEIGHTS

manyChildren.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,ALIGIBILITY,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,1,8866,2.471086e+07
164,1,223,5,1,0,3,306265.282758,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07,1,8866,7.423900e+06
170,1,229,6,1,1,3,564214.583103,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06,1,8866,9.450357e+06
348,1,493,6,1,2,5,242276.000000,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07,1,8866,2.244820e+07
486,1,715,6,1,1,4,641663.724137,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07,1,8866,6.591329e+07


In [27]:
#Calculate, what amount of deficit is left uncovered by social transfer

manyChildren['UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = np.where((manyChildren.MONTHLY_DEFICIT_WEIGHTED > 
                                                               manyChildren.MONTHLY_TRANSFER_WEIGHTED), 
                                                              (manyChildren.MONTHLY_DEFICIT_WEIGHTED - 
                                                               manyChildren.MONTHLY_TRANSFER_WEIGHTED), 0)
manyChildren.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,ALIGIBILITY,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED,UNCOVERED_MONTHLY_DEFICIT_WEIGHTED
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,1,8866,2.471086e+07,0.000000e+00
164,1,223,5,1,0,3,306265.282758,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07,1,8866,7.423900e+06,1.015021e+07
170,1,229,6,1,1,3,564214.583103,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06,1,8866,9.450357e+06,0.000000e+00
348,1,493,6,1,2,5,242276.000000,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07,1,8866,2.244820e+07,6.774550e+07
486,1,715,6,1,1,4,641663.724137,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07,1,8866,6.591329e+07,0.000000e+00


In [28]:
manyChildren

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,ALIGIBILITY,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED,UNCOVERED_MONTHLY_DEFICIT_WEIGHTED
17,1,20,5,1,1,3,7.865611e+05,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,1,8866,2.471086e+07,0.000000e+00
164,1,223,5,1,0,3,3.062653e+05,837.344881,8866,9302,5104.421379,1,1,20987.893103,1.757410e+07,1,8866,7.423900e+06,1.015021e+07
170,1,229,6,1,1,3,5.642146e+05,1065.909930,8866,9302,7836.313654,1,1,8794.118075,9.373738e+06,1,8866,9.450357e+06,0.000000e+00
348,1,493,6,1,2,5,2.422760e+05,2531.942708,8866,9302,3364.944444,1,1,35622.333333,9.019371e+07,1,8866,2.244820e+07,6.774550e+07
486,1,715,6,1,1,4,6.416637e+05,7434.388891,8866,9302,8911.996169,1,1,2340.022989,1.739664e+07,1,8866,6.591329e+07,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51426,92,400,5,1,2,3,1.316199e+06,3764.365077,10375,10503,21936.656705,0,0,0.000000,0.000000e+00,1,10375,3.905529e+07,0.000000e+00
51441,92,422,9,1,0,3,1.375018e+06,6669.362976,10375,10503,12731.644775,1,0,0.000000,0.000000e+00,1,10375,6.919464e+07,0.000000e+00
51767,92,922,5,1,0,3,9.578408e+05,2037.096257,10375,10503,15964.013841,1,0,0.000000,0.000000e+00,1,10375,2.113487e+07,0.000000e+00
51824,92,1003,6,1,1,3,4.610203e+05,2964.275627,10375,10503,6403.060345,1,1,24599.637931,7.292011e+07,1,10375,3.075436e+07,4.216575e+07


In [33]:
print ("Ежемесячный дефицит доходов многодетных семей с детьми до 3-х лет составляет", 
    round(sum(manyChildren.MONTHLY_DEFICIT_WEIGHTED)/10**6), 
      "млн рублей. Общий объём трансфертов равен",
    round(sum(manyChildren.MONTHLY_TRANSFER_WEIGHTED)/10**6), 
       "млн рублей. Пособие на третьих и последующих детей до 3-х лет снижает дефицит доходов на", 
    round((sum(manyChildren.MONTHLY_DEFICIT_WEIGHTED) - sum(manyChildren.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/10**6), 
       "млн рублей, что составляет ", 
    round(100 - sum(manyChildren.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED)/sum(manyChildren.MONTHLY_DEFICIT_WEIGHTED)*100),
       "% от дефицита доходов целевой аудитории и", 
    round((sum(manyChildren.MONTHLY_DEFICIT_WEIGHTED) - 
           sum(manyChildren.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/sum(vndn.MONTHLY_DEFICIT_WEIGHTED)*100, 2), 
       "% от дефицита доходов населения. Всего в России", 
    round(sum(manyChildren.UNDER_2PL*manyChildren.HHOLD_WEIGHTS)),
       "домохозяйств, которые могут получить этот вид пособия, в которых проживают",
    round(sum(manyChildren.UNDER_2PL*manyChildren.HHOLD_SIZE*manyChildren.HHOLD_WEIGHTS)), 
       "человек. Доходы",
    round(sum(manyChildren.UNDER_1PL*manyChildren.HHOLD_WEIGHTS)),
       "домохозяйств (", 
    round(sum(manyChildren.UNDER_1PL * manyChildren.HHOLD_SIZE * manyChildren.HHOLD_WEIGHTS)), " человек) ниже уровня ПМ.")

Ежемесячный дефицит доходов многодетных семей с детьми до 3-х лет составляет 7376 млн рублей. Общий объём трансфертов равен 7852 млн рублей. Пособие на третьих и последующих детей до 3-х лет снижает дефицит доходов на 3627 млн рублей, что составляет  49 % от дефицита доходов целевой аудитории и 7.56 % от дефицита доходов населения. Всего в России 624268 домохозяйств, которые могут получить этот вид пособия, в которых проживают 3439641 человек. Доходы 350885 домохозяйств ( 1966156  человек) ниже уровня ПМ.


### Ежемесячная выплата малоимущим семьям (менее 2 ПМ на 1 члена) на детей от 3-х  до 7-ми лет 

In [34]:
# Define the target audience

threeSeven = vndn[(vndn.UNDER_2PL == 1) & (vndn.FROM_3_TO_7 != 0)]

threeSeven.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED
10,1,13,3,0,1,1,506505.139309,291.919445,8866,9302,14069.587203,1,0,0.000000,0.000000e+00
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00
18,1,21,4,0,1,1,579075.902068,295.302069,8866,9302,12064.081293,1,0,0.000000,0.000000e+00
25,1,38,3,0,1,1,412072.735057,173.750443,8866,9302,11446.464863,1,0,0.000000,0.000000e+00
26,1,40,3,0,1,1,238885.461494,169.157272,8866,9302,6635.707264,1,1,7998.878209,1.353068e+06


In [35]:
#Calculate transfer

threeSeven['MONTHLY_TRANSFER'] = threeSeven.FROM_3_TO_7 * threeSeven.CHILDREN_PL/2

threeSeven.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER
10,1,13,3,0,1,1,506505.139309,291.919445,8866,9302,14069.587203,1,0,0.000000,0.000000e+00,4433.0
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,4433.0
18,1,21,4,0,1,1,579075.902068,295.302069,8866,9302,12064.081293,1,0,0.000000,0.000000e+00,4433.0
25,1,38,3,0,1,1,412072.735057,173.750443,8866,9302,11446.464863,1,0,0.000000,0.000000e+00,4433.0
26,1,40,3,0,1,1,238885.461494,169.157272,8866,9302,6635.707264,1,1,7998.878209,1.353068e+06,4433.0


In [36]:
#Calculate transfer

threeSeven['MONTHLY_TRANSFER_WEIGHTED'] = threeSeven.MONTHLY_TRANSFER * threeSeven.HHOLD_WEIGHTS

threeSeven.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED
10,1,13,3,0,1,1,506505.139309,291.919445,8866,9302,14069.587203,1,0,0.000000,0.000000e+00,4433.0,1.294079e+06
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,4433.0,1.235543e+07
18,1,21,4,0,1,1,579075.902068,295.302069,8866,9302,12064.081293,1,0,0.000000,0.000000e+00,4433.0,1.309074e+06
25,1,38,3,0,1,1,412072.735057,173.750443,8866,9302,11446.464863,1,0,0.000000,0.000000e+00,4433.0,7.702357e+05
26,1,40,3,0,1,1,238885.461494,169.157272,8866,9302,6635.707264,1,1,7998.878209,1.353068e+06,4433.0,7.498742e+05


In [38]:
#Calculate, what amount of deficit is left uncovered by social transfer

threeSeven['UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = 0
threeSeven.loc[threeSeven.MONTHLY_DEFICIT_WEIGHTED > threeSeven.MONTHLY_TRANSFER_WEIGHTED, 
               'UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = (threeSeven.MONTHLY_DEFICIT_WEIGHTED - 
                                                        threeSeven.MONTHLY_TRANSFER_WEIGHTED)

threeSeven.head()

/Users/anastasiia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,MONTHLY_TRANSFER,MONTHLY_TRANSFER_WEIGHTED,UNCOVERED_MONTHLY_DEFICIT_WEIGHTED
10,1,13,3,0,1,1,506505.139309,291.919445,8866,9302,14069.587203,1,0,0.000000,0.000000e+00,4433.0,1.294079e+06,0.000000
17,1,20,5,1,1,3,786561.068965,2787.148739,8866,9302,13109.351149,1,0,0.000000,0.000000e+00,4433.0,1.235543e+07,0.000000
18,1,21,4,0,1,1,579075.902068,295.302069,8866,9302,12064.081293,1,0,0.000000,0.000000e+00,4433.0,1.309074e+06,0.000000
25,1,38,3,0,1,1,412072.735057,173.750443,8866,9302,11446.464863,1,0,0.000000,0.000000e+00,4433.0,7.702357e+05,0.000000
26,1,40,3,0,1,1,238885.461494,169.157272,8866,9302,6635.707264,1,1,7998.878209,1.353068e+06,4433.0,7.498742e+05,603194.229734


In [39]:
print ("Ежемесячный дефицит доходов семей с первыми и вторыми детьми до 3-лет, доход которых ниже 2 региональных ПМ, составляет", 
    round(sum(threeSeven.MONTHLY_DEFICIT_WEIGHTED)/10**6),
        "млн рублей. Общий объём трансфертов равен",
    round(sum(threeSeven.MONTHLY_TRANSFER_WEIGHTED)/10**6), 
       "млн рублей. Пособие на детей от трёх до 7-ми лет снижает дефицит доходов на", 
    round((sum(threeSeven.MONTHLY_DEFICIT_WEIGHTED) - sum(threeSeven.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/10**6), 
       "млн рублей, что составляет ", 
    round(100 - sum(threeSeven.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED)/sum(threeSeven.MONTHLY_DEFICIT_WEIGHTED)*100),
       "% от дефицита доходов целевой аудитории и", 
    round((sum(threeSeven.MONTHLY_DEFICIT_WEIGHTED) - 
           sum(threeSeven.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/sum(vndn.MONTHLY_DEFICIT_WEIGHTED)*100, 2), 
       "% от дефицита доходов населения. Всего в России", 
    round(sum(threeSeven.UNDER_2PL*threeSeven.HHOLD_WEIGHTS)),
       "домохозяйств, которые могут получить этот вид пособия, в которых проживают",
    round(sum(threeSeven.UNDER_2PL*threeSeven.HHOLD_SIZE*threeSeven.HHOLD_WEIGHTS)), 
       "человек. Доходы",
    round(sum(threeSeven.UNDER_1PL*threeSeven.HHOLD_WEIGHTS)),
       "домохозяйств (", 
    round(sum(threeSeven.UNDER_1PL * threeSeven.HHOLD_SIZE * threeSeven.HHOLD_WEIGHTS)), " человек) ниже уровня ПМ.")

Ежемесячный дефицит доходов семей с первыми и вторыми детьми до 3-лет, доход которых ниже 2 региональных ПМ, составляет 19833 млн рублей. Общий объём трансфертов равен 21814 млн рублей. Пособие на детей от трёх до 7-ми лет снижает дефицит доходов на 7193 млн рублей, что составляет  36 % от дефицита доходов целевой аудитории и 15.0 % от дефицита доходов населения. Всего в России 3526791 домохозяйств, которые могут получить этот вид пособия, в которых проживают 15519750 человек. Доходы 1228419 домохозяйств ( 5704712  человек) ниже уровня ПМ.


### Совокупное влияние мер поддержки семей с детьми на дефицит доходов домохозяйств

In [40]:
# Let`s copy the main data table

transfers = vndn.copy()

In [41]:
#Calculate transfer for the first and the second child under 3 years old

transfers['UNDER_3_TRNSF'] = np.where((transfers.UNDER_2PL == 1) & (transfers.UNDER_18 < 3) & (transfers.UNDER_3 != 0),
                                      transfers.UNDER_3 * transfers.CHILDREN_PL*transfers.HHOLD_WEIGHTS, 0)

transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0,0.0


In [46]:
#Define how many children will receive the tranfer

transfers['ALIGIBILITY'] = np.where((vndn.UNDER_18 > 2) & (vndn.UNDER_3 != 0), 
                                    np.where((transfers.UNDER_18 - transfers.UNDER_3) >= 2, 
                                    transfers.UNDER_3,
                                    (np.where((transfers.UNDER_18 - transfers.UNDER_3) == 1, 
                                    (transfers.UNDER_3 - 1), (transfers.UNDER_3 - 2)))), 0)
transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF,ALIGIBILITY
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0,0.0,0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0,0.0,0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0,0.0,0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0,0.0,0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0,0.0,0


In [47]:
#Calculate transfer for families with more than 3 children

transfers['MANY_CHLDRN_TRNSF'] = transfers.ALIGIBILITY * transfers.CHILDREN_PL*transfers.HHOLD_WEIGHTS

transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF,ALIGIBILITY,MANY_CHLDRN_TRNSF
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0,0.0,0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0,0.0,0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0,0.0,0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0,0.0,0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0,0.0,0,0.0


In [48]:
#Calculate transfer for children from 3 to 7 years old

transfers['FROM_3_TO_7_TRNSF'] = np.where((transfers.UNDER_2PL == 1) & (transfers.FROM_3_TO_7 != 0), 
                                          transfers.FROM_3_TO_7 * transfers.CHILDREN_PL/2, 0)

transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF,ALIGIBILITY,MANY_CHLDRN_TRNSF,FROM_3_TO_7_TRNSF
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0,0.0,0,0.0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0,0.0,0,0.0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0,0.0,0,0.0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0,0.0,0,0.0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0,0.0,0,0.0,0.0


In [49]:
#Calculate total transfers amount

transfers['TRNSF_TOTAL'] = (transfers.UNDER_3_TRNSF + 
                            transfers.MANY_CHLDRN_TRNSF + 
                            transfers.FROM_3_TO_7_TRNSF) 

transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,PER_CAPITA_HHOLD_MONTHLY_INCOME,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF,ALIGIBILITY,MANY_CHLDRN_TRNSF,FROM_3_TO_7_TRNSF,TRNSF_TOTAL
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,36182.314569,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,30687.261494,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,16086.095000,1,0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,16000.619167,1,0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,19612.190000,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [50]:
#Calculate, what amount of deficit is left uncovered by social transfer

transfers['UNCOVERED_MONTHLY_DEFICIT_WEIGHTED'] = np.where((transfers.MONTHLY_DEFICIT_WEIGHTED > 
                                                           transfers.TRNSF_TOTAL), 
                                                           (transfers.MONTHLY_DEFICIT_WEIGHTED - 
                                                           transfers.TRNSF_TOTAL), 0)
transfers.head()

,REGION_ID,HHOLD_ID,HHOLD_SIZE,UNDER_3,FROM_3_TO_7,UNDER_18,INCOME,HHOLD_WEIGHTS,CHILDREN_PL,WORKING_AGE_PL,...,UNDER_2PL,UNDER_1PL,MONTHLY_DEFICIT,MONTHLY_DEFICIT_WEIGHTED,UNDER_3_TRNSF,ALIGIBILITY,MANY_CHLDRN_TRNSF,FROM_3_TO_7_TRNSF,TRNSF_TOTAL,UNCOVERED_MONTHLY_DEFICIT_WEIGHTED
0,1,2,2,0,0,0,8.683755e+05,2801.980358,8866,9302,...,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,1,3,3,0,0,1,1.104741e+06,1219.295838,8866,9302,...,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,1,4,2,0,0,0,3.860663e+05,277.577976,8866,9302,...,1,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
3,1,5,2,0,0,0,3.840149e+05,277.577976,8866,9302,...,1,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,1,6,2,0,0,0,4.706926e+05,277.577976,8866,9302,...,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [64]:
print ("Общий объём трансфертов составляет",
    round(sum(transfers.TRNSF_TOTAL)/10**6),
       "млн рублей. ",
    round(transfers[transfers.TRNSF_TOTAL > 0].HHOLD_WEIGHTS.sum()),
       "домохозяйств имеют право на эти пособия. Пособия семьям с детьми снижают дефицит доходов населения на", 
    round((sum(transfers.MONTHLY_DEFICIT_WEIGHTED) - sum(transfers.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED))/10**6), 
       "млн рублей (", 
    round((100 - sum(transfers.UNCOVERED_MONTHLY_DEFICIT_WEIGHTED)/sum(transfers.MONTHLY_DEFICIT_WEIGHTED)*100), 2), 
       "%).")

Общий объём трансфертов составляет 26305 млн рублей.  5025610.0 домохозяйств имеют право на эти пособия. Пособия семьям с детьми снижают дефицит доходов населения на 8542 млн рублей ( 17.81 %).
